In [1]:
# For data manipulation
import pandas as pd

# To extract fundamental data
from bs4 import BeautifulSoup as bs
import requests

# Functions to Parse Data from FinViz

In [2]:
def fundamental_metric(soup, metric):
    return soup.find(text = metric).find_next(class_='snapshot-td2').text

In [3]:
def get_fundamental_data(df):
    for symbol in df.index:
        try:
            url = ("http://finviz.com/quote.ashx?t=" + symbol.lower())
            soup = bs(requests.get(url).content) 
            for m in df.columns:                
                df.loc[symbol,m] = fundamental_metric(soup,m)                
        except Exception as e:
            print (symbol, 'not found')
    return df

# List of Stocks and Ratios You are Interested In

In [4]:
stock_list = ['AMZN','GOOG','PG','KO','IBM','DG','XOM','KO','PEP','MT','NL','GSB','LPL']

metric = ['P/B',
'P/E',
'Forward P/E',
'PEG',
'Debt/Eq',
'EPS (ttm)',
'Dividend %',
'ROE',
'ROI',
'EPS Q/Q',
'Insider Own'
]

# Initialize Pandas DataFrame to Store the Data

In [5]:
df = pd.DataFrame(index=stock_list,columns=metric)
df = get_fundamental_data(df)
df

,P/B,P/E,Forward P/E,PEG,Debt/Eq,EPS (ttm),Dividend %,ROE,ROI,EPS Q/Q,Insider Own
AMZN,18.61,101.14,58.32,3.12,0.67,23.02,-,21.10%,11.70%,12.70%,11.20%
GOOG,4.16,24.74,21.07,1.56,-,49.16,-,-,-,19.30%,5.66%
PG,6.67,66.12,22.97,8.38,0.63,1.81,2.64%,9.60%,4.90%,16.10%,0.10%
KO,10.24,21.94,20.44,4.83,2.25,2.07,3.61%,48.80%,12.20%,134.00%,0.30%
IBM,4.97,10.88,8.73,3.28,3.02,10.73,5.55%,51.70%,11.40%,-5.00%,0.10%
DG,6.76,26.91,21.59,2.49,0.43,6.64,0.81%,25.70%,18.90%,17.00%,0.10%
XOM,0.91,12.86,21.55,2.27,0.25,3.19,8.50%,7.50%,2.30%,-17.30%,0.20%
KO,10.24,21.94,20.44,4.83,2.25,2.07,3.61%,48.80%,12.20%,134.00%,0.30%
PEP,12.34,25.21,21.10,4.56,2.17,5.19,2.92%,51.30%,17.80%,-69.70%,0.10%
MT,0.24,-,6.19,-,0.37,-2.42,2.20%,-6.00%,-2.10%,-259.00%,44.80%


# Remove % Sign and Convert Values to Numeric Type

In [6]:
df['Dividend %'] = df['Dividend %'].str.replace('%', '')
df['ROE'] = df['ROE'].str.replace('%', '')
df['ROI'] = df['ROI'].str.replace('%', '')
df['EPS Q/Q'] = df['EPS Q/Q'].str.replace('%', '')
df['Insider Own'] = df['Insider Own'].str.replace('%', '')
df = df.apply(pd.to_numeric, errors='coerce')
df

,P/B,P/E,Forward P/E,PEG,Debt/Eq,EPS (ttm),Dividend %,ROE,ROI,EPS Q/Q,Insider Own
AMZN,18.61,101.14,58.32,3.12,0.67,23.02,NaN,21.1,11.7,12.7,11.20
GOOG,4.16,24.74,21.07,1.56,NaN,49.16,NaN,NaN,NaN,19.3,5.66
PG,6.67,66.12,22.97,8.38,0.63,1.81,2.64,9.6,4.9,16.1,0.10
KO,10.24,21.94,20.44,4.83,2.25,2.07,3.61,48.8,12.2,134.0,0.30
IBM,4.97,10.88,8.73,3.28,3.02,10.73,5.55,51.7,11.4,-5.0,0.10
DG,6.76,26.91,21.59,2.49,0.43,6.64,0.81,25.7,18.9,17.0,0.10
XOM,0.91,12.86,21.55,2.27,0.25,3.19,8.50,7.5,2.3,-17.3,0.20
KO,10.24,21.94,20.44,4.83,2.25,2.07,3.61,48.8,12.2,134.0,0.30
PEP,12.34,25.21,21.10,4.56,2.17,5.19,2.92,51.3,17.8,-69.7,0.10
MT,0.24,NaN,6.19,NaN,0.37,-2.42,2.20,-6.0,-2.1,-259.0,44.80


# Filter Good Companies

### 1. Companies which are quoted at low valuations
P/E < 15 and P/B < 1

In [7]:
df_filtered = df[(df['P/E'].astype(float)<15) & (df['P/B'].astype(float) < 1)]
df_filtered

,P/B,P/E,Forward P/E,PEG,Debt/Eq,EPS (ttm),Dividend %,ROE,ROI,EPS Q/Q,Insider Own
XOM,0.91,12.86,21.55,2.27,0.25,3.19,8.50,7.5,2.3,-17.3,0.20
NL,0.42,4.94,NaN,NaN,0.00,0.53,6.11,8.5,-0.7,78.2,82.98


### 2. Further filter companies which have demonstrated earning power 
EPS Q/Q > 10%

In [8]:
df_filtered = df_filtered[df_filtered['EPS Q/Q'].astype(float) > 10]
df_filtered

,P/B,P/E,Forward P/E,PEG,Debt/Eq,EPS (ttm),Dividend %,ROE,ROI,EPS Q/Q,Insider Own
NL,0.42,4.94,NaN,NaN,0.0,0.53,6.11,8.5,-0.7,78.2,82.98


### Management having substantial ownership in the business
Insider Own > 30%

In [9]:
df = df[df['Insider Own'].astype(float) > 30]
df

,P/B,P/E,Forward P/E,PEG,Debt/Eq,EPS (ttm),Dividend %,ROE,ROI,EPS Q/Q,Insider Own
MT,0.24,NaN,6.19,NaN,0.37,-2.42,2.20,-6.0,-2.1,-259.0,44.80
NL,0.42,4.94,NaN,NaN,0.00,0.53,6.11,8.5,-0.7,78.2,82.98
LPL,0.34,NaN,NaN,NaN,1.20,-3.23,NaN,-21.7,-10.9,NaN,51.10
